In [4]:
!pip install tensorflow

   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
    --------------------------------------- 5.5/376.0 MB 30.5 MB/s eta 0:00:13
   - -------------------------------------- 13.6/376.0 MB 34.3 MB/s eta 0:00:11
   -- ------------------------------------- 22.3/376.0 MB 37.1 MB/s eta 0:00:10
   --- ------------------------------------ 30.1/376.0 MB 36.8 MB/s eta 0:00:10
   --- ------------------------------------ 34.6/376.0 MB 33.3 MB/s eta 0:00:11
   ---- ----------------------------------- 39.3/376.0 MB 31.2 MB/s eta 0:00:11
   ---- ----------------------------------- 44.3/376.0 MB 30.3 MB/s eta 0:00:11
   ----- ---------------------------------- 49.3/376.0 MB 29.3 MB/s eta 0:00:12
   ----- ---------------------------------- 54.5/376.0 MB 28.9 MB/s eta 0:00:12
   ------ --------------------------------- 60.0/376.0 MB 28.5 MB/s eta 0:00:12
   ------ --------------------------------- 65.3/376.0 MB 28.3 MB/s eta 0:00:11
   ------- -------------------------------- 70.5/3

In [19]:
import os
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import SGD

# ─── Config ──────────────────────────────────────────────────────────
FOLDS_DIR = "cv_folds"
TIMESTEPS = 10
FEATURE   = "PTS"
N_JOBS    = 4

# hyperparameter grid (you can expand this list)
hyperparams = [
    {"units": 32, "epochs": 200},
]

# ─── Model builder ───────────────────────────────────────────────────
def make_lstm_model(timesteps, n_feats, units):
    model = Sequential([
        LSTM(units, input_shape=(timesteps, n_feats), name="lstm"),
        Dense(1, name="output")
    ])
    opt = SGD(learning_rate=0.01, momentum=0.9, decay=0.001)
    model.compile(optimizer=opt, loss="mse")
    return model

# ─── Sequence builder ────────────────────────────────────────────────
def build_sequences(series: np.ndarray):
    X, y = [], []
    for i in range(TIMESTEPS, len(series)):
        X.append(series[i-TIMESTEPS:i].reshape(TIMESTEPS, 1))
        y.append(series[i])
    return np.array(X), np.array(y)

# ─── Evaluate one fold ───────────────────────────────────────────────
def eval_lstm_fold(fold_idx, hp):
    # load
    tr = pd.read_csv(os.path.join(FOLDS_DIR, f"fold_{fold_idx}_train.csv"),
                     parse_dates=["GAME_DATE"]).sort_values("GAME_DATE")
    te = pd.read_csv(os.path.join(FOLDS_DIR, f"fold_{fold_idx}_test.csv"),
                     parse_dates=["GAME_DATE"]).sort_values("GAME_DATE")

    # build train/test series
    s_tr = tr[FEATURE].values
    s_te = np.concatenate([s_tr[-TIMESTEPS:], te[FEATURE].values])

    X_tr, y_tr = build_sequences(s_tr)
    X_te, y_te = build_sequences(s_te)

    # scale
    scaler = StandardScaler().fit(X_tr.reshape(-1, 1))
    X_tr = scaler.transform(X_tr.reshape(-1, 1)).reshape(-1, TIMESTEPS, 1)
    X_te = scaler.transform(X_te.reshape(-1, 1)).reshape(-1, TIMESTEPS, 1)

    # train
    model = make_lstm_model(TIMESTEPS, 1, hp["units"])
    model.fit(X_tr, y_tr,
              epochs=hp["epochs"],
              batch_size=32,
              verbose=0)

    # predict + metrics
    pred = model.predict(X_te, verbose=0).flatten()
    mse = mean_squared_error(y_te, pred)
    mae = mean_absolute_error(y_te, pred)
    r2  = r2_score(y_te, pred)

    return {
        "PTS_rmse": np.sqrt(mse),
        "PTS_mae": mae,
        "PTS_r2":  r2
    }

# ─── Run CV & aggregate ────────────────────────────────────────────────
all_dfs = []
for hp in hyperparams:
    # parallel over folds
    metrics_list = Parallel(n_jobs=N_JOBS)(
        delayed(eval_lstm_fold)(fold, hp) for fold in range(1, 11)
    )
    df = pd.DataFrame(metrics_list)
    df["fold"]   = np.arange(1, len(df)+1)
    df["units"]  = hp["units"]
    df["epochs"] = hp["epochs"]
    all_dfs.append(df)

# concatenate + reorder
final_df = pd.concat(all_dfs, ignore_index=True)
cols = ["units", "epochs", "fold"] + [c for c in final_df.columns if c not in ("units","epochs","fold")]
final_df = final_df[cols]

print(final_df)


   units  epochs  fold  PTS_rmse   PTS_mae    PTS_r2
0     32     200     1  9.495587  7.494048 -0.091186
1     32     200     2  8.881021  7.023573 -0.040305
2     32     200     3  8.671586  7.061141 -0.043636
3     32     200     4  9.064177  7.208680 -0.031926
4     32     200     5  8.902708  7.140062 -0.025884
5     32     200     6  9.169643  7.292805 -0.021336
6     32     200     7  8.744066  7.018606 -0.001855
7     32     200     8  9.122160  7.193718 -0.005051
8     32     200     9  8.492743  6.815726 -0.032622
9     32     200    10  8.964492  7.095685 -0.002431
